# Module 04: Experimental Design

Welcome to Module 04! Now that you have a research question and hypothesis, let's learn how to **design rigorous experiments** to test them.

## What You'll Learn

- Understanding different types of variables
- Designing controlled experiments
- Randomization and control groups
- Sample size and statistical power
- Common experimental designs (A/B testing, factorial designs)
- Threats to validity and how to avoid them

## Prerequisites

- Completed Modules 00-03
- Understanding of basic statistics

## Time Required

**40 minutes**

---

In [ ]:
# ========================================
# Setup
# ========================================

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
output_dir = "outputs/notebook_04"
os.makedirs(output_dir, exist_ok=True)

print("✅ Setup complete!")
print(f"Output directory: {output_dir}")

## Part 1: Understanding Variables

In experimental research, understanding variables is crucial for proper design.

### Types of Variables

#### 1. Independent Variable (IV)

**Definition**: The variable you manipulate or change

**Also called**: 
- Predictor variable
- Treatment variable
- Explanatory variable

**Examples**:
- Algorithm type (LSTM vs ARIMA)
- Feature set (with/without RFM features)
- Training data size
- Learning rate

**Key point**: This is what you control in the experiment

#### 2. Dependent Variable (DV)

**Definition**: The variable you measure (the outcome)

**Also called**:
- Outcome variable
- Response variable
- Measured variable

**Examples**:
- Prediction accuracy
- F1-score
- Training time
- User satisfaction

**Key point**: This changes in response to the IV

#### 3. Confounding Variables

**Definition**: Variables that might affect the DV but aren't the IV

**Also called**:
- Lurking variables
- Third variables

**Examples**:
- Data quality (when comparing algorithms)
- Hardware differences (when measuring speed)
- Researcher experience
- Time of day for data collection

**Key point**: These can lead to false conclusions if not controlled

#### 4. Control Variables

**Definition**: Confounding variables that you **hold constant**

**Examples**:
- Use same dataset for all algorithms
- Use same hardware for all tests
- Use same evaluation metrics

**Key point**: By controlling these, you ensure fair comparison

### Example: Stock Price Prediction Experiment

**Research Question**: Does LSTM outperform ARIMA for stock prediction?

- **Independent Variable (IV)**: Algorithm (LSTM vs ARIMA)
- **Dependent Variable (DV)**: Prediction accuracy (RMSE)
- **Confounding Variables**: 
  - Stock volatility
  - Time period
  - Feature selection
  - Data quality
- **Control Variables** (what we hold constant):
  - Same stocks
  - Same time period
  - Same features
  - Same evaluation metric

In [ ]:
# ========================================
# Visualize Relationship Between Variables
# ========================================

# Simulate an experiment: Effect of training data size on accuracy
# IV: Training data size
# DV: Model accuracy
# Confounding: We'll show what happens if not controlled

# Scenario 1: Properly controlled (same data quality)
data_sizes = np.array([100, 500, 1000, 2000, 5000, 10000])
accuracy_controlled = 0.5 + 0.3 * np.log(data_sizes) / np.log(10000)
accuracy_controlled += np.random.normal(0, 0.02, len(data_sizes))  # Small random noise

# Scenario 2: Confounded (data quality varies)
# Worse quality data for larger sizes (unrealistic but illustrative)
data_quality_effect = -0.15 * (data_sizes / 10000)
accuracy_confounded = accuracy_controlled + data_quality_effect

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Controlled experiment
axes[0].plot(data_sizes, accuracy_controlled, "o-", color="green", linewidth=2, markersize=8)
axes[0].set_xlabel("Training Data Size", fontweight="bold")
axes[0].set_ylabel("Accuracy", fontweight="bold")
axes[0].set_title(
    "Controlled Experiment\n(Data quality held constant)", fontweight="bold", color="green"
)
axes[0].grid(alpha=0.3)
axes[0].set_ylim([0.4, 0.9])

# Confounded experiment
axes[1].plot(data_sizes, accuracy_confounded, "o-", color="red", linewidth=2, markersize=8)
axes[1].set_xlabel("Training Data Size", fontweight="bold")
axes[1].set_ylabel("Accuracy", fontweight="bold")
axes[1].set_title(
    "Confounded Experiment\n(Data quality varies - WRONG!)", fontweight="bold", color="red"
)
axes[1].grid(alpha=0.3)
axes[1].set_ylim([0.4, 0.9])

plt.tight_layout()
plt.savefig(f"{output_dir}/controlled_vs_confounded.png", dpi=150, bbox_inches="tight")
print(f"✅ Visualization saved to: {output_dir}/controlled_vs_confounded.png")
plt.show()

print("\n💡 KEY LESSON:")
print("Left: Proper control shows expected relationship (more data = better)")
print("Right: Confounding variable masks true effect!")
print("\nAlways control confounding variables!")

## Part 2: Experimental Design Principles

Good experiments follow these core principles:

### 1. Randomization

**What**: Randomly assign subjects/samples to different groups

**Why**: 
- Eliminates selection bias
- Balances unknown confounds across groups
- Allows for statistical inference

**Example**: When testing a new algorithm
- ❌ **Bad**: Test on easy data samples, compare to hard samples
- ✅ **Good**: Randomly split data into train/test, apply both algorithms

### 2. Control Group

**What**: A baseline group for comparison

**Why**: Shows what would happen without the treatment

**Example**: Testing a new feature engineering technique
- **Treatment group**: Model with new features
- **Control group**: Model with original features
- **Compare**: Difference in performance

### 3. Replication

**What**: Repeat the experiment multiple times

**Why**:
- Reduces impact of random variation
- Increases confidence in results
- Helps detect real effects vs noise

**Example**: 
- Run model training 10 times with different random seeds
- Report mean and standard deviation
- Not just one lucky run!

### 4. Blinding

**What**: Keeping participants/researchers unaware of group assignment

**Types**:
- **Single-blind**: Participants don't know their group
- **Double-blind**: Neither participants nor researchers know

**Why**: Prevents bias in measurement or behavior

**Example in ML**:
- Have someone else evaluate model outputs
- Don't tell them which algorithm produced which results
- Prevents confirmation bias

### 5. Sample Size

**What**: Number of observations/samples

**Why**: 
- Too small: Can't detect real effects
- Too large: Wastes resources
- Just right: Adequate statistical power

**How to determine**: Statistical power analysis (see Part 4)

## Part 3: Common Experimental Designs

### Design 1: Simple Randomized Experiment (A/B Test)

**Structure**:
```
Random Assignment
       ↓
   ┌───┴───┐
   A       B
   ↓       ↓
Measure  Measure
   ↓       ↓
   Compare
```

**When to use**: Comparing two conditions

**Example**: 
- Group A: Algorithm 1
- Group B: Algorithm 2
- Compare accuracy

**Pros**: Simple, easy to analyze
**Cons**: Can only test one factor at a time

### Design 2: Factorial Design

**Structure**: Test multiple factors simultaneously

**Example**: 2×2 Factorial (2 factors, 2 levels each)
```
Factor A: Algorithm (LSTM vs ARIMA)
Factor B: Features (Basic vs Advanced)

4 Conditions:
1. LSTM + Basic
2. LSTM + Advanced
3. ARIMA + Basic
4. ARIMA + Advanced
```

**When to use**: Want to test interactions between factors

**Pros**: 
- Test multiple factors efficiently
- Can detect interactions

**Cons**: 
- More complex
- Larger sample size needed

### Design 3: Before-After (Within-Subjects)

**Structure**: Measure same subjects before and after treatment

**Example**:
- Measure model performance with current features
- Add new features
- Measure performance again
- Compare difference

**Pros**: Controls for individual differences
**Cons**: Order effects, can't "unlearn" what was learned

### Design 4: Crossover Design

**Structure**: Each group gets both treatments in different order

```
Period 1      Period 2
Group 1: A  →  B
Group 2: B  →  A
```

**When to use**: Want benefits of within-subjects but need to control order

**Pros**: Controls for order effects
**Cons**: More complex, time-consuming

### Design 5: Stratified Random Assignment

**Structure**: Divide into strata, then randomize within each

**Example**:
```
Stratify by dataset size:
- Small datasets: Randomly assign to A or B
- Medium datasets: Randomly assign to A or B  
- Large datasets: Randomly assign to A or B
```

**When to use**: Important confounds you want to balance

**Pros**: Ensures balance on key variables
**Cons**: Need to know important strata in advance

In [ ]:
# ========================================
# Demonstrate Factorial Design
# ========================================

# Simulate a 2x2 factorial experiment
# Factor A: Algorithm (Random Forest vs XGBoost)
# Factor B: Feature Engineering (Basic vs Advanced)

np.random.seed(42)

# Simulate accuracy scores for each condition
results = {
    "Algorithm": ["Random Forest"] * 50
    + ["XGBoost"] * 50
    + ["Random Forest"] * 50
    + ["XGBoost"] * 50,
    "Features": ["Basic"] * 100 + ["Advanced"] * 100,
    "Accuracy": (
        list(np.random.normal(0.75, 0.05, 50))  # RF + Basic
        + list(np.random.normal(0.78, 0.05, 50))  # XGB + Basic
        + list(np.random.normal(0.82, 0.05, 50))  # RF + Advanced
        + list(np.random.normal(0.88, 0.05, 50))  # XGB + Advanced (interaction!)
    ),
}

df_factorial = pd.DataFrame(results)

# Analyze
print("FACTORIAL EXPERIMENT RESULTS")
print("=" * 60)
print("\nMean Accuracy by Condition:")
print(df_factorial.groupby(["Algorithm", "Features"])["Accuracy"].mean())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Bar chart
summary = df_factorial.groupby(["Algorithm", "Features"])["Accuracy"].mean().reset_index()
x = np.arange(len(summary))
colors = ["lightblue", "blue", "lightcoral", "red"]
bars = axes[0].bar(x, summary["Accuracy"], color=colors, alpha=0.7, edgecolor="black")
axes[0].set_xticks(x)
axes[0].set_xticklabels(
    [f"{row['Algorithm']}\n{row['Features']}" for _, row in summary.iterrows()], fontsize=9
)
axes[0].set_ylabel("Accuracy", fontweight="bold")
axes[0].set_title("2×2 Factorial Design Results", fontweight="bold", fontsize=12)
axes[0].set_ylim([0.6, 1.0])
axes[0].grid(axis="y", alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[0].text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.3f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# Plot 2: Interaction plot
for algo in ["Random Forest", "XGBoost"]:
    data = summary[summary["Algorithm"] == algo]
    axes[1].plot([0, 1], data["Accuracy"], "o-", linewidth=2, markersize=10, label=algo)

axes[1].set_xticks([0, 1])
axes[1].set_xticklabels(["Basic", "Advanced"])
axes[1].set_xlabel("Features", fontweight="bold")
axes[1].set_ylabel("Accuracy", fontweight="bold")
axes[1].set_title(
    "Interaction Plot\n(Non-parallel lines = interaction!)", fontweight="bold", fontsize=12
)
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].set_ylim([0.7, 0.95])

plt.tight_layout()
plt.savefig(f"{output_dir}/factorial_design.png", dpi=150, bbox_inches="tight")
print(f"\n✅ Factorial design plot saved to: {output_dir}/factorial_design.png")
plt.show()

print("\n💡 KEY INSIGHT:")
print("The non-parallel lines show an INTERACTION:")
print("XGBoost benefits MORE from advanced features than Random Forest does!")
print("This insight would be missed with separate A/B tests!")

## Part 4: Sample Size and Statistical Power

### What is Statistical Power?

**Statistical Power**: Probability of detecting an effect when it truly exists

**Formula**: Power = 1 - β (where β = probability of Type II error)

**Conventional target**: 0.80 (80% power)

### The Four Key Concepts

1. **Effect Size**: How big is the difference you expect?
   - Small: Cohen's d ≈ 0.2
   - Medium: Cohen's d ≈ 0.5
   - Large: Cohen's d ≈ 0.8

2. **Sample Size (n)**: How many observations?
   - What we usually want to calculate

3. **Significance Level (α)**: Probability of Type I error
   - Usually 0.05

4. **Power (1-β)**: Probability of detecting real effect
   - Usually aim for 0.80

**Key Relationship**: If you know any 3, you can calculate the 4th!

### Sample Size Calculation

**General principle**: 
- Smaller expected effect → Larger sample needed
- Higher power desired → Larger sample needed
- Lower α (more stringent) → Larger sample needed

### Rule of Thumb (for t-tests)

| Effect Size | Sample Size per Group (α=0.05, power=0.80) |
|-------------|-------------------------------------------|
| Small (d=0.2) | ~400 |
| Medium (d=0.5) | ~64 |
| Large (d=0.8) | ~26 |

In [ ]:
# ========================================
# Sample Size Calculator
# ========================================


def calculate_sample_size_ttest(effect_size, power=0.8, alpha=0.05):
    """
    Calculate required sample size for independent t-test.

    Args:
        effect_size (float): Cohen's d
        power (float): Desired statistical power (default 0.8)
        alpha (float): Significance level (default 0.05)

    Returns:
        int: Required sample size per group
    """
    # This is a simplified calculation
    # For precise calculations, use statsmodels or G*Power

    from scipy.stats import norm

    z_alpha = norm.ppf(1 - alpha / 2)  # Two-tailed
    z_beta = norm.ppf(power)

    n = 2 * ((z_alpha + z_beta) / effect_size) ** 2

    return int(np.ceil(n))


# Calculate for different effect sizes
print("SAMPLE SIZE CALCULATOR")
print("=" * 60)
print(f"Settings: α = 0.05, Power = 0.80\n")

effect_sizes = {"Small (0.2)": 0.2, "Medium (0.5)": 0.5, "Large (0.8)": 0.8}

results = []
for label, d in effect_sizes.items():
    n = calculate_sample_size_ttest(d)
    total = n * 2  # Both groups
    results.append({"Effect Size": label, "n per group": n, "Total n": total})
    print(f"Effect Size {label:15s}: {n:4d} per group, {total:4d} total")

# Visualize relationship
effect_range = np.linspace(0.1, 1.0, 50)
sample_sizes = [calculate_sample_size_ttest(d) for d in effect_range]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(effect_range, sample_sizes, linewidth=2, color="steelblue")
ax.fill_between(effect_range, sample_sizes, alpha=0.3, color="steelblue")

# Mark common effect sizes
for label, d in effect_sizes.items():
    n = calculate_sample_size_ttest(d)
    ax.plot(d, n, "ro", markersize=10)
    ax.annotate(
        f"{label}\nn={n}",
        xy=(d, n),
        xytext=(d + 0.05, n + 50),
        fontweight="bold",
        fontsize=9,
        arrowprops=dict(arrowstyle="->", color="red", lw=1.5),
    )

ax.set_xlabel("Effect Size (Cohen's d)", fontweight="bold", fontsize=11)
ax.set_ylabel("Required Sample Size (per group)", fontweight="bold", fontsize=11)
ax.set_title("Sample Size vs Effect Size\n(α=0.05, Power=0.80)", fontweight="bold", fontsize=13)
ax.grid(alpha=0.3)
ax.set_ylim([0, max(sample_sizes) + 100])

plt.tight_layout()
plt.savefig(f"{output_dir}/sample_size_calculation.png", dpi=150, bbox_inches="tight")
print(f"\n✅ Sample size plot saved to: {output_dir}/sample_size_calculation.png")
plt.show()

print("\n💡 KEY LESSON:")
print("Smaller effects require MUCH larger samples to detect!")
print("Always calculate sample size BEFORE collecting data!")

## Part 5: Threats to Validity

Even well-designed experiments can have problems. Here are common threats:

### Internal Validity Threats

*Does the IV really cause changes in the DV?*

#### 1. Selection Bias
**Problem**: Groups differ before treatment
**Solution**: Random assignment

#### 2. History
**Problem**: External events affect results
**Example**: Software update during experiment
**Solution**: Control timing, use control group

#### 3. Maturation
**Problem**: Participants change over time
**Example**: Model improves just due to more training time
**Solution**: Control group experiences same time

#### 4. Testing Effects
**Problem**: Being measured changes behavior
**Example**: Model overfits to validation set from repeated testing
**Solution**: Hold out true test set

#### 5. Instrumentation
**Problem**: Measurement tool changes
**Example**: Updating evaluation metric mid-experiment
**Solution**: Keep measurement consistent

### External Validity Threats

*Do results generalize beyond this experiment?*

#### 1. Sample Bias
**Problem**: Sample not representative
**Example**: Only test on clean, balanced data
**Solution**: Use diverse, realistic datasets

#### 2. Setting
**Problem**: Lab conditions differ from real world
**Example**: Test on small datasets, deploy on massive scale
**Solution**: Test at realistic scale

#### 3. Time
**Problem**: Results specific to time period
**Example**: Model tested in stable market, deployed in volatile one
**Solution**: Test across different conditions

### How to Protect Validity

**Checklist**:
- ☑ Random assignment
- ☑ Control group
- ☑ Adequate sample size
- ☑ Control confounding variables
- ☑ Blind evaluation when possible
- ☑ Replicate experiments
- ☑ Test on diverse datasets
- ☑ Report all analyses (don't cherry-pick)
- ☑ Pre-register hypotheses when possible

In [ ]:
# ========================================
# Create Experimental Design Checklist
# ========================================

checklist = """
EXPERIMENTAL DESIGN CHECKLIST
==============================

PLANNING PHASE
□ Research question clearly defined
□ Hypothesis stated (including direction)
□ Independent variable(s) identified
□ Dependent variable(s) identified
□ Confounding variables identified
□ Control variables determined
□ Experimental design chosen (A/B, factorial, etc.)
□ Sample size calculated (power analysis)
□ Randomization method determined
□ Control group established

EXECUTION PHASE
□ Random seed set (for reproducibility)
□ Data randomized properly
□ Groups balanced on key variables
□ Control variables held constant
□ Blinding implemented (if applicable)
□ Data collection consistent across groups
□ All procedures documented
□ Unexpected events recorded

ANALYSIS PHASE
□ Check assumptions (normality, etc.)
□ Appropriate statistical test chosen
□ Multiple comparison correction (if needed)
□ Effect sizes reported (not just p-values)
□ Confidence intervals included
□ All analyses reported (not selective)
□ Sensitivity analyses conducted

VALIDITY CHECKS
□ Internal validity threats addressed
□ External validity considered
□ Generalizability discussed
□ Limitations acknowledged
□ Alternative explanations considered

REPORTING
□ Methods described in detail
□ Sample size justified
□ Randomization described
□ All conditions reported
□ Negative results included
□ Code/data shared (when possible)
□ Reproducibility information provided
"""

# Save checklist
with open(f"{output_dir}/experimental_design_checklist.txt", "w") as f:
    f.write(checklist)

print(checklist)
print(f"\n✅ Checklist saved to: {output_dir}/experimental_design_checklist.txt")
print("\nUse this checklist for every experiment you design!")

---

## Summary

Congratulations on completing Module 04!

### Key Takeaways

✅ **Variables**: Understand IV (manipulate), DV (measure), confounds (control)

✅ **Design Principles**: Randomization, control groups, replication, adequate sample size

✅ **Common Designs**: A/B tests, factorial, before-after, crossover, stratified

✅ **Statistical Power**: Calculate sample size before collecting data

✅ **Validity**: Protect against internal and external validity threats

### What You Can Do Now

- Identify and control variables in experiments
- Choose appropriate experimental designs
- Calculate required sample sizes
- Design factorial experiments to test interactions
- Recognize and address threats to validity

### Practice Exercise

**Exercise**: Design Your Own Experiment

Using your research question from Module 03:

1. Identify your IV, DV, and potential confounds
2. Choose an appropriate experimental design
3. List control variables
4. Calculate required sample size
5. Fill out the experimental design checklist
6. Identify potential threats to validity

Save this - you'll implement it in Module 09!

---

### Up Next

In **Module 05: Data Collection Methods**, you'll learn:
- Primary vs secondary data
- Data collection techniques
- Ensuring data quality
- Data validation strategies
- Documentation during collection

---

**Ready to continue?** Move on to `05_data_collection_methods.ipynb`!

**Need to review?** Go back to any challenging section.